In [3]:
using Pkg
Pkg.activate("./")
using ParametricOperators

  Activating new project at `~/Documents/Research/ParametricOperators.jl/src`
  No Changes to `~/Documents/Research/ParametricOperators.jl/src/Project.toml`
  No Changes to `~/Documents/Research/ParametricOperators.jl/src/Manifest.toml`


LoadError: ArgumentError: Package ParametricOperators not found in current path.
- Run `import Pkg; Pkg.add("ParametricOperators")` to install the ParametricOperators package.

In [2]:

function TuckerCompress(size::Vector,rank::Vector,restriction::Vector)
    factors = []
    n = length(size)
    
    core = ParMatrix(T,rank[1],prod(rank[2:n]))
    push!(factors,ParMatrix(T,size[1],rank[1]))
    for i  = 2:length(size)
        push!(factors,ParMatrix(T,rank[i],size[i]))
    end
    return ParTucker(core,factors,restriction)
end

T = Float32
i = 5; b = 3; nx = 6; ny = 6; nt = 6; o = 5
mx = 2; my = 2; mt = 2
layer = 1
rank = [2,2,2,2,2]
tensor_size = [o,i,2*mx,2*my,mt]
restr = [2,2,2]
tensor  = TuckerCompress(tensor_size,rank,restr)


LoadError: UndefVarError: `ParMatrix` not defined

In [4]:
d = ParametricOperators.to_Dict(tensor)

Dict{String, Any} with 6 entries:
  "rank"        => Any[2, 2, 2, 2, 2]
  "shape"       => Any[5, 5, 4, 4, 2]
  "T"           => "Float32"
  "id"          => "UUID:2929d969-4bda-4681-8f7c-b8ee01ac850f"
  "restriction" => [2, 2, 2]
  "type"        => "ParTucker"

In [8]:
Type{ParTucker}

Type{ParTucker}

In [5]:
ParametricOperators.from_Dict(typeof(tensor),d)

ParTucker{Float32, 5}(ParMatrix{Float32}(2, 16, UUID("17233c94-c815-42b9-9092-9f9b1655bf3f")), Any[ParMatrix{Float32}(5, 2, UUID("07ea5cfa-f625-43b2-abc9-09e727ef2690")), ParMatrix{Float32}(2, 5, UUID("541e8c23-5601-4a34-a171-9f05bdcb7186")), ParMatrix{Float32}(2, 4, UUID("e649fc93-627b-4a37-9136-efe89d8f5944")), ParMatrix{Float32}(2, 4, UUID("0f1fa39f-7bc6-4f0d-b8e0-49dc6bab1739")), ParMatrix{Float32}(2, 2, UUID("e18a12da-b617-4749-b6d7-58b5acd28ece"))], [2, 2, 2], UUID("2929d969-4bda-4681-8f7c-b8ee01ac850f"))

LoadError: MethodError: no method matching collect()

[0mClosest candidates are:
[0m  collect([91m::LLVM.FunctionParameterSet[39m)
[0m[90m   @[39m [35mLLVM[39m [90m~/.julia/packages/LLVM/TBM0O/src/core/[39m[90m[4mfunction.jl:110[24m[39m
[0m  collect([91m::DataStructures.SparseIntSet[39m)
[0m[90m   @[39m [36mDataStructures[39m [90m~/.julia/packages/DataStructures/jFDPC/src/[39m[90m[4msparse_int_set.jl:201[24m[39m
[0m  collect([91m::LLVM.StructTypeElementSet[39m)
[0m[90m   @[39m [35mLLVM[39m [90m~/.julia/packages/LLVM/TBM0O/src/core/[39m[90m[4mtype.jl:255[24m[39m
[0m  ...


In [4]:
TuckerShape(tensor)

LoadError: UndefVarError: `TuckerShape` not defined

In [12]:
using Parameters

@with_kw struct ModelConfig
    nx::Int = 64
    ny::Int = 64
    nt::Int = 51
    nc_in::Int = 4
    nc_mid::Int = 128
    nc_lift::Int = 20
    nc_out::Int = 1
    mx::Int = 4
    my::Int = 4
    mt::Int = 4
    nblocks::Int = 1
    dtype::DataType = Float32
    partition::Vector{Int} = [1, 4] 
    TuckerRank::Vector{Int} = [2,2,2,2,2,2] ##need to adjust this
end

ModelConfig

In [22]:
modconfig = ModelConfig()

ModelConfig
  nx: Int64 64
  ny: Int64 64
  nt: Int64 51
  nc_in: Int64 4
  nc_mid: Int64 128
  nc_lift: Int64 20
  nc_out: Int64 1
  mx: Int64 4
  my: Int64 4
  mt: Int64 4
  nblocks: Int64 1
  dtype: Float32 <: AbstractFloat
  partition: Array{Int64}((2,)) [1, 4]
  TuckerRank: Array{Int64}((6,)) [2, 2, 2, 2, 2, 2]


In [19]:
function TuckerCompress(size::Vector,rank::Vector,restriction::Vector)
    factors = []
    n = length(size)
    
    core = ParMatrix(T,rank[1],prod(rank[2:n]))
    push!(factors,ParMatrix(T,size[1],rank[1]))
    for i  = 2:length(size)
        push!(factors,ParMatrix(T,rank[i],size[i]))
    end
    return ParTucker(core,factors,restriction)
end

64

In [2]:
function TuckerCompress(size::Vector,rank::Vector,restriction::Vector)
    factors = []
    n = length(size)
    
    core = ParMatrix(T,rank[1],prod(rank[2:n]))
    push!(factors,ParMatrix(T,size[1],rank[1]))
    for i  = 2:length(size)
        push!(factors,ParMatrix(T,rank[i],size[i]))
    end
    return ParTucker(core,factors,restriction)
end
T = Float32
i = 5; b = 3; nx = 6; ny = 6; nt = 6; o = 5
mx = 2; my = 2; mt = 2
layer = 1
rank = [2,2,2,2,2]
tensor_size = [o,i,2*mx,2*my,mt]
restr = [2,2,2]
tensor  = TuckerCompress(tensor_size,rank,restr)
tensor(2)

LoadError: UndefVarError: ParMatrix not defined

In [4]:
mutable struct Model
    config::ModelConfig
    lifts::ParKron
    convs::Vector
    sconvs::Vector
    biases::Vector
    sconv_biases::Vector
    projects::Vector
    weight_mixes::Vector

    #first, function to get Tucker factors (ParMatrix)


    function Model(config::ModelConfig)

        T = config.dtype
        
        sconvs = []
        convs = []
        projects = []
        sconv_biases = []
        biases = []
        weight_mixes = []
    
        weight_shape = [config.nc_lift, config.nc_lift, 2*config.mx, 2*config.my,
        config.mt,config.nblocks]
        weight_mix = TuckerCompress(weight_shape,config.TuckerRank,
        [config.mx, config.my, config.mt])
        push!(weight_mixes, weight_mix)

        function spectral_convolution(layer::Int)
    
            # Build 3D Fourier transform with real-valued FFT along time
            fourier_x = ParDFT(Complex{T}, config.nx)
            fourier_y = ParDFT(Complex{T}, config.ny)
            fourier_t = ParDFT(T, config.nt)
    
            # Build restrictions to low-frequency modes
            restrict_x = ParRestriction(Complex{T}, Range(fourier_x), [1:config.mx, config.nx-config.mx+1:config.nx])
            restrict_y = ParRestriction(Complex{T}, Range(fourier_y), [1:config.my, config.ny-config.my+1:config.ny])
            restrict_t = ParRestriction(Complex{T}, Range(fourier_t), [1:config.mt])
            
            restrict_dft = ParKron((restrict_y * fourier_y) ⊗ (restrict_x * fourier_x), (restrict_t * fourier_t) ⊗ ParIdentity(T, config.nc_lift))
    
            sconv = restrict_dft' * weight_mix(layer) * restrict_dft
    
            return sconv
        end
    
        # Lift Channel dimension
        lifts = ParKron(ParIdentity(T,config.ny) ⊗ ParIdentity(T,config.nx), ParIdentity(T,config.nt) ⊗ ParMatrix(T, config.nc_lift, config.nc_in, "ParMatrix_LIFTS:(1)"))
        bias = ParBroadcasted(ParMatrix(T, config.nc_lift, 1, "ParMatrix_BIAS:(1)"))
    
        # lifts = distribute(lifts, config.partition)
    
        push!(biases, bias)
    
        for i in 1:config.nblocks
    
            sconv_layer = spectral_convolution(i)
            conv_layer = ParKron(ParIdentity(T,config.ny) ⊗ ParIdentity(T,config.nx), ParIdentity(T,config.nt) ⊗ ParMatrix(T, config.nc_lift, config.nc_lift, "ParMatrix_SCONV:($(i))"))
            bias = ParBroadcasted(ParMatrix(T, config.nc_lift, 1, "ParMatrix_SCONV:($(i))"))
    
            # conv_layer = distribute(conv_layer, config.partition)
    
            push!(sconv_biases, bias)
            push!(sconvs, sconv_layer)
            push!(convs, conv_layer)
        end
    
        # Uplift channel dimension once more
        uc = ParKron(ParIdentity(T,config.ny) ⊗ ParIdentity(T,config.nx), ParIdentity(T,config.nt) ⊗ ParMatrix(T, config.nc_mid, config.nc_lift, "ParMatrix_LIFTS:(2)"))
        bias = ParBroadcasted(ParMatrix(T, config.nc_mid, 1, "ParMatrix_BIAS:(2)"))
    
        uc = distribute(uc, config.partition)
    
        push!(biases, bias)
        push!(projects, uc)
    
        # Project channel dimension
        pc = ParKron(ParIdentity(T,config.ny) ⊗ ParIdentity(T,config.nx), ParIdentity(T,config.nt) ⊗ ParMatrix(T, config.nc_out, config.nc_mid, "ParMatrix_LIFTS:(3)"))
        bias = ParBroadcasted(ParMatrix(T, config.nc_out, 1, "ParMatrix_BIAS:(3)"))
    
        pc = distribute(pc, config.partition)
    
        push!(biases, bias)
        push!(projects, pc)
    
        new(config, lifts, convs, sconvs, biases, sconv_biases, projects, weight_mixes)
    end
end

In [ ]:
mutable struct Model
    config::ModelConfig
    lifts::ParKron
    convs::Vector
    sconvs::Vector
    biases::Vector
    sconv_biases::Vector
    projects::Vector
    weight_mixes::Vector

    #first, function to get Tucker factors (ParMatrix)


    function Model(config::ModelConfig)

        T = config.dtype
        
        sconvs = []
        convs = []
        projects = []
        sconv_biases = []
        biases = []
        weight_mixes = []
    
        weight_shape = [config.nc_lift, config.nc_lift, 2*config.mx, 2*config.my,
        config.mt,config.nblocks]
        weight_mix = TuckerCompress(weight_shape,config.TuckerRank,
        [config.mx, config.my, config.mt])
        push!(weight_mixes, weight_mix)

        function spectral_convolution(layer::Int)
    
            # Build 3D Fourier transform with real-valued FFT along time
            fourier_x = ParDFT(Complex{T}, config.nx)
            fourier_y = ParDFT(Complex{T}, config.ny)
            fourier_t = ParDFT(T, config.nt)
    
            # Build restrictions to low-frequency modes
            restrict_x = ParRestriction(Complex{T}, Range(fourier_x), [1:config.mx, config.nx-config.mx+1:config.nx])
            restrict_y = ParRestriction(Complex{T}, Range(fourier_y), [1:config.my, config.ny-config.my+1:config.ny])
            restrict_t = ParRestriction(Complex{T}, Range(fourier_t), [1:config.mt])
            
            restrict_dft = ParKron((restrict_y * fourier_y) ⊗ (restrict_x * fourier_x), (restrict_t * fourier_t) ⊗ ParIdentity(T, config.nc_lift))
    
            sconv = restrict_dft' * weight_mix(layer) * restrict_dft
    
            return sconv
        end
    
        # Lift Channel dimension
        lifts = ParKron(ParIdentity(T,config.ny) ⊗ ParIdentity(T,config.nx), ParIdentity(T,config.nt) ⊗ ParMatrix(T, config.nc_lift, config.nc_in, "ParMatrix_LIFTS:(1)"))
        bias = ParBroadcasted(ParMatrix(T, config.nc_lift, 1, "ParMatrix_BIAS:(1)"))
    
        # lifts = distribute(lifts, config.partition)
    
        push!(biases, bias)
    
        for i in 1:config.nblocks
    
            sconv_layer = spectral_convolution(i)
            conv_layer = ParKron(ParIdentity(T,config.ny) ⊗ ParIdentity(T,config.nx), ParIdentity(T,config.nt) ⊗ ParMatrix(T, config.nc_lift, config.nc_lift, "ParMatrix_SCONV:($(i))"))
            bias = ParBroadcasted(ParMatrix(T, config.nc_lift, 1, "ParMatrix_SCONV:($(i))"))
    
            # conv_layer = distribute(conv_layer, config.partition)
    
            push!(sconv_biases, bias)
            push!(sconvs, sconv_layer)
            push!(convs, conv_layer)
        end
    
        # Uplift channel dimension once more
        uc = ParKron(ParIdentity(T,config.ny) ⊗ ParIdentity(T,config.nx), ParIdentity(T,config.nt) ⊗ ParMatrix(T, config.nc_mid, config.nc_lift, "ParMatrix_LIFTS:(2)"))
        bias = ParBroadcasted(ParMatrix(T, config.nc_mid, 1, "ParMatrix_BIAS:(2)"))
    
        uc = distribute(uc, config.partition)
    
        push!(biases, bias)
        push!(projects, uc)
    
        # Project channel dimension
        pc = ParKron(ParIdentity(T,config.ny) ⊗ ParIdentity(T,config.nx), ParIdentity(T,config.nt) ⊗ ParMatrix(T, config.nc_out, config.nc_mid, "ParMatrix_LIFTS:(3)"))
        bias = ParBroadcasted(ParMatrix(T, config.nc_out, 1, "ParMatrix_BIAS:(3)"))
    
        pc = distribute(pc, config.partition)
    
        push!(biases, bias)
        push!(projects, pc)
    
        new(config, lifts, convs, sconvs, biases, sconv_biases, projects, weight_mixes)
    end
end